<a href="https://colab.research.google.com/github/ChiaraDeMartin95/ciao/blob/master/PionIDwithML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "PionID"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
#os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [13]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')
path_to_dir='/content/gdrive/My\ Drive/FianlExamML/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
!ls /content/gdrive/My\ Drive/FianlExamML

ls: cannot access '/content/gdrive/My Drive/FianlExamML': No such file or directory


In [16]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')
path_to_dir='/content/gdrive/My\ Drive/FinalExamML/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import uproot

ModuleNotFoundError: ignored

In [18]:
!pip install uproot 

     |████████████████████████████████| 122kB 4.5MB/s 
     |████████████████████████████████| 92kB 7.8MB/s 


In [19]:
import uproot

In [24]:
filePion=uproot.open("/content/gdrive/My Drive/AnalysisResults.root")

In [23]:
!ls /content/gdrive/My\ Drive/Ana*

'/content/gdrive/My Drive/AnalysisResults.root'


In [25]:
filePion.keys()

[b'MultSelection_MC;1', b'MyTaskPions;1']

In [26]:
dirPion=filePion["MyTaskPions"]

In [27]:
dirPion.keys()

[b'MyOutputContainer;1',
 b'fSignalTree;1',
 b'fBkgTree;1',
 b'MyOutputContainer3;1',
 b'MyOutputContainer4;1',
 b'Risoluzione;1']

In [28]:
treePion=dirPion["fSignalTree"]

In [29]:
treePion.keys()

[b'fTreeVariableNSigmaTOFPion',
 b'fTreeVariableNSigmaTOFProton',
 b'fTreeVariableNSigmaTOFKaon',
 b'fTreeVariableNSigmaTPCPion',
 b'fTreeVariableNSigmaTPCProton',
 b'fTreeVariableNSigmaTPCKaon',
 b'fTreeVariabledEdX',
 b'fTreeVariableTOFBeta',
 b'fTreeVariableTOFMass',
 b'fTreeVariablePtTrigger',
 b'fTreeVariableChargeTrigger',
 b'fTreeVariableEtaTrigger',
 b'fTreeVariablePhiTrigger',
 b'fTreeVariableDCAz',
 b'fTreeVariableDCAxy',
 b'fTreeVariableisPrimaryTrigger',
 b'fTreeVariableMultiplicity',
 b'fTreeVariablePDGCodeTrigger']

In [30]:
treePion['fTreeVariableChargeTrigger'].array()

array([ 1,  1,  1, ..., -1,  1,  1], dtype=int32)

In [31]:
treePion.pandas.df(["fTreeVariableNSigmaTOF*", "fTreeVariableNSigmaTPC*", "fTreeVariabledEdX", "fTreeVariableTOFMass", "fTreeVariablePtTrigger"])

,fTreeVariableNSigmaTOFPion,fTreeVariableNSigmaTOFProton,fTreeVariableNSigmaTOFKaon,fTreeVariableNSigmaTPCPion,fTreeVariableNSigmaTPCProton,fTreeVariableNSigmaTPCKaon,fTreeVariabledEdX,fTreeVariableTOFMass,fTreeVariablePtTrigger
entry,,,,,,,,,
0,0.918567,-35.513672,-20.527399,0.323678,-11.748958,-2.683780,50.31250,0.169956,0.680152
1,-999.000000,-999.000000,-999.000000,-1.153996,-17.027597,-15.634204,57.09375,-999.000000,0.265325
2,-999.000000,-999.000000,-999.000000,0.155855,-15.890946,-14.801109,53.15625,-999.000000,0.270509
3,1.226982,-31.313883,-32.581455,-0.590945,-16.512909,-8.736343,51.81250,0.150573,0.501138
4,-999.000000,-999.000000,-999.000000,1.670261,-14.866197,-14.983491,59.18750,-999.000000,0.243150
...,...,...,...,...,...,...,...,...,...
32269,1.221653,-35.550972,-15.472705,-1.598596,-7.957152,-2.154730,52.12500,0.077342,0.838925
32270,-999.000000,-999.000000,-999.000000,-0.429741,-6.021661,-0.481019,43.56250,-999.000000,0.865467
32271,-999.000000,-999.000000,-999.000000,0.598376,-10.698731,-3.459585,44.75000,-999.000000,0.494926
